In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='-1'
import jax 
import pickle
import matplotlib.pyplot as plt

from phyOT.utils import *

In [ ]:
from config import *

file = 'data/model.pkl'
with open(file, 'rb') as f:
    model1 = pickle.load(f)
    
params_fno = model1['params_fno']
cfg = model1['cfg']
fno = FNO(cfg.FNO, FNO_utils2D)
import types
fno.forward = types.MethodType(forward, fno)

file_data = 'data/data.pkl'

with open(file_data, 'rb') as f:
    data = pickle.load(f)

Us    = jax.vmap(fno.forward, in_axes=(None, 0))(params_fno,  data['dataZs'])
solver.device='cpu'
UsSolver = jax.vmap(solver.solve, in_axes=(None, 0, None))(f_field*0., data['dataZs'], f_field)

Us = Us.reshape(data['dataZs'].shape[0], -1)
UsSolver = UsSolver.reshape(data['dataZs'].shape[0], -1)

relative_error = jnp.mean( jnp.linalg.norm(Us - UsSolver, axis=-1) / jnp.linalg.norm(UsSolver, axis=-1))
print('relative_error:', relative_error * 100.)

grid = data['grid']
loss = model1['aux']['aux'][:,2]
plt.semilogy(loss)
plt.grid()
plt.show()

for i in [2]:
    plt.contourf(grid[0], grid[1], UsSolver[i].reshape(grid[0].shape), 50)
    plt.colorbar()
    plt.show()
    
    plt.contourf(grid[0], grid[1], Us[i].reshape(grid[0].shape), 50)
    plt.colorbar()
    plt.show()